In [1]:
%load_ext autoreload
%autoreload 2

In [157]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from skimage.data import lena

from skimage.filters.rank import equalize
from skimage.morphology import disk

from nolearn_utils.iterators import (
    make_iterator,
    BufferedBatchIteratorMixin,
    LCNBatchIteratorMixin,
    AffineTransformBatchIteratorMixin,
    ShuffleBatchIteratorMixin
)

from joblib import Parallel, delayed

In [16]:
# def parallelize_map(func, arr, n_jobs=1, *args, **kwargs):
#     return Parallel(n_jobs)(delayed(func)(x, *args, **kwargs) for x in arr)

# arr = np.arange(0, 10)
# def multiply(x, factor=2):
#     return x * factor
# parallelize_map(multiply, arr)

In [207]:
y = np.repeat([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 3, 4], 100)
X = np.arange(len(y))
target_size = np.bincount(y).min()
mask = np.zeros_like(y, dtype=bool)
for yval in np.unique(y):
    mask[random.sample(np.where(y == yval)[0], target_size)] = True

In [261]:
class RebalanceBatchIteratorMixin(object):
    def __call__(self, X, y):
        super(RebalanceBatchIteratorMixin, self).__call__(X, y)
        assert y.ndim == 1
        X, y = self.X, self.y
        target_size = np.bincount(y).min()
        mask = np.zeros_like(y, dtype=bool)
        for yval in np.unique(y):
            mask[random.sample(np.where(y == yval)[0], target_size)] = True
        self.X, self.y = X[mask], y[mask]

In [235]:
mixins = [
    RebalanceBatchIteratorMixin,
    ShuffleBatchIteratorMixin
]

Iterator = make_iterator('Iterator', mixins)

# x = np.expand_dims(lena(), 3)
# X = np.tile(x, 64).transpose(3, 2, 0, 1).astype(float) / 255
# y = np.random.random(64) * 100
# y = y.astype(np.int32)

y = np.repeat([0, 0, 0, 0, 1, 1, 2, 2, 3, 4], 500)
X = np.arange(len(y))
# print X.shape, y.shape

iterator = Iterator(
    batch_size=16,
#     n_jobs=8,
#     affine_p=1,
#     affine_scale_choices=np.linspace(0.75, 1.25, 5),
#     affine_translation_choices=np.arange(-24, 24, 4),
#     affine_rotation_choices=np.arange(-90, 90, 5),
)
iterator(X, y)
gen = iter(iterator)

In [259]:
iterator.y

array([1, 3, 1, ..., 1, 2, 1])

In [256]:
%%time
Xb, yb = gen.next()

print Xb
print yb
print np.bincount(yb)

[4612  447 4762   75 4456 4154 3044  735 4260 4014 2911  243 4551 4526   28
 3373]
[4 0 4 0 3 3 2 0 3 3 1 0 4 4 0 2]
[5 1 2 4 4]
CPU times: user 2.18 ms, sys: 0 ns, total: 2.18 ms
Wall time: 1.07 ms
